In [1]:
import tensorflow as tf

##### skip connection

- help with gradient vanishing problem
- identity block
    - input dimension = output dimension
    <img src="../images/identity block.png" style="width:650px;height:150px;">
- convolutional block
    - input dimension != output dimension
    - shortcut CONV2D reduces the dimension of the input to match up for the addition step
    <img src="../images/convolutional block.png" style="width:650px;height:150px;">


In [2]:
def identity_block(X, f, filters, initializer=tf.keras.initializers.random_uniform):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = tf.keras.layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=1, padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization()(X) # Default axis
    X = tf.keras.layers.Activation('relu')(X)
    
    X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=1, padding='same', kernel_initializer=initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization()(X) 
    X = tf.keras.layers.Activation('relu')(X) 
    
    X = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=1, padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization()(X) 
    
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)

    return X

In [3]:
def convolutional_block(X, f, filters, s=2, initializer=tf.keras.initializers.glorot_uniform):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = tf.keras.layers.Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same', kernel_initializer=initializer(seed=0))(X) 
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X) 

    X = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X) 
    X = tf.keras.layers.BatchNormalization()(X)
    
    X_shortcut = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization()(X_shortcut)

    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X

##### ResNet50  

<img src="../images/resnet.png" style="width:850px;height:150px;">

In [4]:
def ResNet50(input_shape=(64, 64, 3), classes=6, training=False):
    # Define the input as a tensor with shape input_shape
    X_input = tf.keras.layers.Input(input_shape)

    # Zero-Padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = tf.keras.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    ## Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2) 
    X = identity_block(X, 3, [128, 128, 512]) 
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024],s=2) 
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2) 
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL
    X = tf.keras.layers.AveragePooling2D(pool_size=(2,2), padding='same')(X) 

    # output layer
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    
    model = tf.keras.models.Model(inputs = X_input, outputs = X)

    return model

model = ResNet50(input_shape=(64, 64, 3), classes=6)